# Author John Payne
# Step 1
* Generate a response using the Casseopeia wrapper

In [1]:
import requests as re
import time

def getAPI_key():
    f = open("../api_key.txt", "r")
    return f.read()
api_key = getAPI_key()

In [229]:
import json
def KeyMap_summoner_spells():
    
    #Matches key values of champions with their names, useful later in the script
    with open('data/summoner_spell_map.txt', 'r', encoding="utf8") as summonerList_file:
        summonerList = json.load(summonerList_file)
        summonerList_file.close()
        summonerList = summonerList['data']
    
    SummonerNames = []
    SummonerKeys = []
    for value in summonerList:
        SummonerNames.append(value)
        SummonerKeys.append(summonerList[value]['key'])
        
    mapped_summoners = dict(zip(SummonerKeys,SummonerNames))    
    return mapped_summoners

In [5]:
def check_status():    
    url = 'https://na1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
    header_dict = {
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "X-Riot-Token": api_key
    }
    

    return re.get(url, headers = header_dict).status_code

In [6]:
check_status()

200

In [7]:
def get_challenger_player_stats():
    url = 'https://na1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
    header_dict = {
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "X-Riot-Token": api_key
    }

    
    return re.get(url, headers = header_dict).json()

In [8]:
challenger_ladder = get_challenger_player_stats()

In [9]:
def get_challenger_player_id(player_id):
    
    #Using league_v4 
    url = 'https://na1.api.riotgames.com/lol/summoner/v4/summoners/' + player_id
    header_dict = {
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "X-Riot-Token": api_key
    }
    

        
    return re.get(url, headers = header_dict).json()

In [10]:
def get_matches(puuid, startTime_Unix, endTime_Unix):
    url = 'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/' + puuid + '/ids'
    header_dict = {
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "X-Riot-Token": api_key
    }
    
    params_dict = {
        #Only picking games played on patch 12.15, 08-10-2022 to 0-24-2022
        'startTime' : startTime_Unix,
        'endTime' : endTime_Unix,
        
        #Only picking ranked 5v5 games
        'queue' : 420,
        'type' : 'ranked',
        
        #Only grabbing the first 50 results
        'start' : 0,
        'count' : 50
    }
    return re.get(url, headers = header_dict, params = params_dict).json()

In [11]:
class Summoner:
    
    def __init__(self, summonerId, startTime_Unix, endTime_Unix):
        self.summonerId = summonerId
        
        #Looping through the ladder until a match is found with the input summoner id
        for value in challenger_ladder['entries']:
            if value['summonerId'] == self.summonerId:
                
                #Once correct player is found assign lp, wins, losses, winstreak, and ign to the player
                self.leaguepoints = value['leaguePoints']
                self.wins = value['wins']
                self.losses = value['losses']
                self.hotStreak = value['hotStreak']
                self.summonerName = value['summonerName']
        
        #Finds all the accounts of the player in dictionary format
        self.accounts = get_challenger_player_id(summonerId)
        #Finds the puuid of the player
        self.puuid = self.accounts['puuid']
        #Finds the players summoner level
        self.summonerLevel = self.accounts['summonerLevel']
        
        #Returns a list of matches the player has played based on the parameters in the get_matches function
        self.matches = get_matches(self.puuid, startTime_Unix, endTime_Unix)
        
        #Total games played by this summoner on this specific patch
        self.total_matches = len(self.matches)
        
    

In [12]:
player_info = {}
for i in range(2):
    temp = 'summoner' + str(i)
    player_info[temp] = Summoner(challenger_ladder['entries'][i]['summonerId'], 1660125600, 1661335200)
    print(i)
    time.sleep(1)

0
1


In [13]:
def get_match_info(match):
    url = 'https://americas.api.riotgames.com/lol/match/v5/matches/' + match
    header_dict = {
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "X-Riot-Token": api_key
    }
    
    return re.get(url, headers = header_dict).json()

In [197]:
class Match_details:
    
    def __init__(self, Match_id):
        self.Match_id = Match_id
        
        #Calling matchesV5 api
        self.call = get_match_info(Match_id)
        
        #Storing api call and filtering out uneeded data
        self.Match_info = self.call['info']
        
        #list of dictionaries including stats for each player
        self.players = self.Match_info['participants']
        
        #Tracking team information
        self.team1 = self.Match_info['teams'][0]
        self.team2 = self.Match_info['teams'][1]
        
        #Storing winning and losing team
        self.team1_win = self.team1['win']
        self.team2_win = self.team2['win']
        
        #Storing bans for each team
        self.team1_bans = self.team1['bans']
        self.team2_bans = self.team1['bans']
      
        #Creating a list of encrypted summoner ids
        self.summoner_Id = []
        for i, player in enumerate(self.players):
            self.summoner_Id.append(player['summonerId'])
        
        #Creating a player class to hold data on each player in the game
        class Player:
    
            def __init__(self, summoner_Id, players):
                self.summoner_Id = summoner_Id
                # Holds lane, champion and team of given player
                self.players = players
                for value in self.players:
                    if value['summonerId'] == self.summoner_Id:
                        self.Lane = value['individualPosition']
                        self.championId = value['championId'] 
                        self.teamId = value['teamId']
                        self.summonerName = value['summonerName']
                        self.summonerLevel = value['summonerLevel']
                        if self.teamId == 100:
                            self.teamId = 'team1'
                        elif self.teamId == 200:
                            self.teamId = 'team2'
                #Grabbing winrate/winstreak/rank/lp of the given player        
                self.player_stats = get_player_stats(self.summoner_Id)
                for queue in self.player_stats:
                    print(queue)
                    if queue['queueType'] == 'RANKED_SOLO_5x5':
                        self.wins = queue['wins']
                        self.losses = queue['losses']
                        self.tier = queue['tier']
                        self.rank = queue['rank']
                        self.lp = queue['leaguePoints']
                        self.veteran = queue['veteran']
                        self.winstreak = queue['hotStreak']
                    else:
                        continue
        
        #Creating a dictionary to hold all the player information (lane, champion, team)
        self.player_position = {}
        for i, summoner_Id in enumerate(self.summoner_Id):
            temp = 'Player' + str(i)
            self.player_position[temp] = Player(summoner_Id, self.players)
            time.sleep(20)
            

In [198]:
#Getting matches for each player listed in player_info
games = []
for key in player_info:
    games.append(player_info[key].matches)

In [199]:
#Joining together the sublists (games of each player)
total_games = []
for a in range(len(games)):
    for b in range (len(games[a])):
        total_games.append(games[a][b])

In [200]:
#Finding unique games
total_games = list(set(total_games))
print(len(total_games))

60


In [201]:
#API request to get player winrates etc.
def get_player_stats(summoner_id):
    url = 'https://na1.api.riotgames.com/lol/league/v4/entries/by-summoner/' + summoner_id
    header_dict = {
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "X-Riot-Token": api_key
    }
    return re.get(url, headers = header_dict).json()

In [202]:
%%time
games = {}
for i, game in enumerate(total_games[0:5]):  
    temp = 'Game' + str(i)
    games[temp] = Match_details(game)
    print(i)
    time.sleep(1)

{'leagueId': '697ed7b8-6d1d-3295-a91a-ad04b9a7d1eb', 'queueType': 'RANKED_SOLO_5x5', 'tier': 'GRANDMASTER', 'rank': 'I', 'summonerId': '5sRvtnhtU34Hx7cMVBV5IdQtk8wD_17WG5TiPY_SXYl06PQ', 'summonerName': 'Cytheur', 'leaguePoints': 686, 'wins': 775, 'losses': 749, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}
{'leagueId': 'b36bb689-d115-467a-bd21-9c08834ebc4d', 'queueType': 'RANKED_FLEX_SR', 'tier': 'PLATINUM', 'rank': 'I', 'summonerId': '5sRvtnhtU34Hx7cMVBV5IdQtk8wD_17WG5TiPY_SXYl06PQ', 'summonerName': 'Cytheur', 'leaguePoints': 18, 'wins': 13, 'losses': 9, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}
{'leagueId': '697ed7b8-6d1d-3295-a91a-ad04b9a7d1eb', 'queueType': 'RANKED_SOLO_5x5', 'tier': 'GRANDMASTER', 'rank': 'I', 'summonerId': 'OqLfVa9DeAZibYG5niO0FUoG1AizU3napuq5W2eL9RWM8z0', 'summonerName': 'Chase', 'leaguePoints': 499, 'wins': 305, 'losses': 276, 'veteran': False, 'inactive': False, 'freshBlood': True, 'hotStreak'

TypeError: string indices must be integers

In [157]:
Games = []
Games.append(games['Game4'].Match_info['gameId'])
Games.append(games['Game4'].Match_info['gameVersion'])


Player_detail_from_game = []
for player in Player_list:
    Players = []
    Players.append(games['Game4'].Match_info['gameId'])
    Players.append(games['Game4'].player_position[player].summonerName)
    Players.append(games['Game4'].player_position[player].wins)
    Players.append(games['Game4'].player_position[player].losses)
    Players.append(games['Game4'].player_position[player].Lane)
    Players.append(games['Game4'].player_position[player].championId)
    Players.append(games['Game4'].player_position[player].tier)
    Players.append(games['Game4'].player_position[player].rank)
    Players.append(games['Game4'].player_position[player].lp)
    Players.append(games['Game4'].player_position[player].veteran)
    Players.append(games['Game4'].player_position[player].winstreak)
    Players.append(games['Game4'].player_position[player].summonerLevel)
    Player_detail_from_game.append(Players)

In [183]:
x = Match_details('NA1_4409917779')

{'leagueId': '697ed7b8-6d1d-3295-a91a-ad04b9a7d1eb', 'queueType': 'RANKED_SOLO_5x5', 'tier': 'GRANDMASTER', 'rank': 'I', 'summonerId': '5sRvtnhtU34Hx7cMVBV5IdQtk8wD_17WG5TiPY_SXYl06PQ', 'summonerName': 'Cytheur', 'leaguePoints': 686, 'wins': 775, 'losses': 749, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}
{'leagueId': 'b36bb689-d115-467a-bd21-9c08834ebc4d', 'queueType': 'RANKED_FLEX_SR', 'tier': 'PLATINUM', 'rank': 'I', 'summonerId': '5sRvtnhtU34Hx7cMVBV5IdQtk8wD_17WG5TiPY_SXYl06PQ', 'summonerName': 'Cytheur', 'leaguePoints': 18, 'wins': 13, 'losses': 9, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}
{'leagueId': '697ed7b8-6d1d-3295-a91a-ad04b9a7d1eb', 'queueType': 'RANKED_SOLO_5x5', 'tier': 'GRANDMASTER', 'rank': 'I', 'summonerId': 'OqLfVa9DeAZibYG5niO0FUoG1AizU3napuq5W2eL9RWM8z0', 'summonerName': 'Chase', 'leaguePoints': 499, 'wins': 305, 'losses': 276, 'veteran': False, 'inactive': False, 'freshBlood': True, 'hotStreak'

In [210]:
total_games[0:5]

['NA1_4409917779',
 'NA1_4410425992',
 'NA1_4411968972',
 'NA1_4410361359',
 'NA1_4414782727']

In [237]:
t = KeyMap_summoner_spells()
t['21']

'SummonerBarrier'

In [299]:
def KeyMap_Champions():
    
    #Matches key values of champions with their names, useful later in the script
    with open('data/champion_map.txt', 'r', encoding="utf8") as champList_file:
        champList = json.load(champList_file)
        champList_file.close()
        champList = champList['data']
    
    champNames = []
    champKeys = []
    for value in champList:
        champNames.append(value)
        champKeys.append(champList[value]['key'])
        
    mapped_champions = dict(zip(champKeys, champNames))    
    return mapped_champions

In [322]:
p = KeyMap_Champions()
p

{'266': 'Aatrox',
 '103': 'Ahri',
 '84': 'Akali',
 '166': 'Akshan',
 '12': 'Alistar',
 '32': 'Amumu',
 '34': 'Anivia',
 '1': 'Annie',
 '523': 'Aphelios',
 '22': 'Ashe',
 '136': 'AurelionSol',
 '268': 'Azir',
 '432': 'Bard',
 '200': 'Belveth',
 '53': 'Blitzcrank',
 '63': 'Brand',
 '201': 'Braum',
 '51': 'Caitlyn',
 '164': 'Camille',
 '69': 'Cassiopeia',
 '31': 'Chogath',
 '42': 'Corki',
 '122': 'Darius',
 '131': 'Diana',
 '119': 'Draven',
 '36': 'DrMundo',
 '245': 'Ekko',
 '60': 'Elise',
 '28': 'Evelynn',
 '81': 'Ezreal',
 '9': 'Fiddlesticks',
 '114': 'Fiora',
 '105': 'Fizz',
 '3': 'Galio',
 '41': 'Gangplank',
 '86': 'Garen',
 '150': 'Gnar',
 '79': 'Gragas',
 '104': 'Graves',
 '887': 'Gwen',
 '120': 'Hecarim',
 '74': 'Heimerdinger',
 '420': 'Illaoi',
 '39': 'Irelia',
 '427': 'Ivern',
 '40': 'Janna',
 '59': 'JarvanIV',
 '24': 'Jax',
 '126': 'Jayce',
 '202': 'Jhin',
 '222': 'Jinx',
 '145': 'Kaisa',
 '429': 'Kalista',
 '43': 'Karma',
 '30': 'Karthus',
 '38': 'Kassadin',
 '55': 'Katarina',


In [321]:
x.Match_info

{'gameCreation': 1660907907195,
 'gameDuration': 1506,
 'gameEndTimestamp': 1660909453885,
 'gameId': 4409917779,
 'gameMode': 'CLASSIC',
 'gameName': 'teambuilder-match-4409917779',
 'gameStartTimestamp': 1660907947352,
 'gameType': 'MATCHED_GAME',
 'gameVersion': '12.15.458.1416',
 'mapId': 11,
 'participants': [{'assists': 3,
   'baronKills': 0,
   'basicPings': 3,
   'bountyLevel': 0,
   'challenges': {'12AssistStreakCount': 0,
    'abilityUses': 172,
    'acesBefore15Minutes': 0,
    'alliedJungleMonsterKills': 1.5,
    'baronTakedowns': 0,
    'blastConeOppositeOpponentCount': 0,
    'bountyGold': 150,
    'buffsStolen': 0,
    'completeSupportQuestInTime': 0,
    'controlWardsPlaced': 0,
    'damagePerMinute': 427.51870664241574,
    'damageTakenOnTeamPercentage': 0.21468686343391175,
    'dancedWithRiftHerald': 0,
    'deathsByEnemyChamps': 4,
    'dodgeSkillShotsSmallWindow': 0,
    'doubleAces': 0,
    'dragonTakedowns': 0,
    'earlyLaningPhaseGoldExpAdvantage': 0,
    'effe

In [319]:
for team in x.Match_info['teams']:
    for ban in team['bans']:
        current_ban = str(ban['championId'])
        print(type(current_ban))
        try: 
            current_ban = p[current_ban]
            print(current_ban)
        except:
            print('null')

<class 'str'>
MasterYi
<class 'str'>
Fiora
<class 'str'>
Graves
<class 'str'>
Kayn
<class 'str'>
Draven
<class 'str'>
Yone
<class 'str'>
Kayn
<class 'str'>
Trundle
<class 'str'>
null
<class 'str'>
Yuumi


In [327]:
x.player_position

{'Player0': <__main__.Match_details.__init__.<locals>.Player at 0x2c4729f9640>,
 'Player1': <__main__.Match_details.__init__.<locals>.Player at 0x2c473273d30>,
 'Player2': <__main__.Match_details.__init__.<locals>.Player at 0x2c47207b340>,
 'Player3': <__main__.Match_details.__init__.<locals>.Player at 0x2c472335f70>,
 'Player4': <__main__.Match_details.__init__.<locals>.Player at 0x2c47343eb50>,
 'Player5': <__main__.Match_details.__init__.<locals>.Player at 0x2c4734229a0>,
 'Player6': <__main__.Match_details.__init__.<locals>.Player at 0x2c473c52670>,
 'Player7': <__main__.Match_details.__init__.<locals>.Player at 0x2c4733eba30>,
 'Player8': <__main__.Match_details.__init__.<locals>.Player at 0x2c473c52730>,
 'Player9': <__main__.Match_details.__init__.<locals>.Player at 0x2c4733eba00>}

In [331]:
total_games[0:2]

['NA1_4409917779', 'NA1_4410425992']

In [360]:
final = [
    [4403325301, '12.15.458.1416', 1538],
    [
        [4403325301, 'Huo Guo Xiao Zi', 94, 88, 'TOP', 'GRANDMASTER', 'I', 524, False, False, 139, 'SummonerFlash', 'SummonerHaste', 'Annie'],
        [4403325301, 'jamaican banana', 128, 104, 'JUNGLE', 'MASTER', 'I', 407, True, False, 84, 'SummonerFlash', 'SummonerSmite', 'Ivern'],
        [4403325301, 'Fishlord', 467, 419, 'MIDDLE', 'CHALLENGER', 'I', 1105, True, False, 570, 'SummonerFlash', 'SummonerHaste', 'Swain'],
        [4403325301, 'ControlMageEGirl', 181, 165, 'BOTTOM', 'GRANDMASTER', 'I', 662, False, False, 169, 'SummonerFlash', 'SummonerHeal', 'Jhin'],
        [4403325301, 'A D', 117, 91, 'UTILITY', 'GRANDMASTER', 'I', 772, False, False, 651, 'SummonerFlash', 'SummonerExhaust', 'Senna'],
        [4403325301, 'Augustine', 151, 124, 'TOP', 'CHALLENGER', 'I', 884, False, False, 368, 'SummonerHaste', 'SummonerTeleport', 'Gwen'],
        [4403325301, 'Dragowski', 239, 217, 'JUNGLE', 'MASTER', 'I', 82, False, False, 139, 'SummonerSmite', 'SummonerFlash', 'Diana'],
        [4403325301, 'MasKa', 155, 132, 'MIDDLE', 'GRANDMASTER', 'I', 518, True, False, 60, 'SummonerFlash', 'SummonerDot', 'Akshan'],
        [4403325301, 'spam tab', 245, 184, 'BOTTOM', 'CHALLENGER', 'I', 1395, True, False, 292, 'SummonerFlash', 'SummonerHeal', 'Aphelios'],
        [4403325301, 'Twitch BTangLoL', 144, 117, 'UTILITY', 'GRANDMASTER', 'I', 538, False, True, 312, 'SummonerFlash', 'SummonerDot', 'Neeko']
    ],
    [
        ['Annie', 2, 7, 6, False, 144, 10, 19], 
        ['Ivern', 3, 3, 14, False, 21, 71, 16], 
        ['Swain', 6, 4, 5, False, 183, 46, 14],
        ['Jhin', 9, 4, 12, False, 202, 2, 22],
        ['Senna', 4, 6, 16, False, 34, 13, 77],
        ['Gwen', 7, 2, 1, True, 242, 19, 22],
        ['Diana', 3, 7, 7, True, 17, 50, 14],
        ['Akshan', 4, 6, 8, True, 159, 32, 14],
        ['Aphelios', 6, 4, 8, True, 211, 22, 29],
        ['Neeko', 4, 5, 8, True, 39, 17, 55]      
    ],
    [
        4403325301, False,
        ['Garen', 'Sylas', 'Kindred', 'Aatrox', 'Draven'],
        ['Kalista', 'Zed', 'Fiora', 'Qiyana', 'Yuumi']   
    ]
]
    
Games = [4403325301, '12.15.458.1416', 1538]
Players = [
        [4403325301, 'Huo Guo Xiao Zi', 94, 88, 'TOP', 'GRANDMASTER', 'I', 524, False, False, 139, 'SummonerFlash', 'SummonerHaste', 'Annie'],
        [4403325301, 'jamaican banana', 128, 104, 'JUNGLE', 'MASTER', 'I', 407, True, False, 84, 'SummonerFlash', 'SummonerSmite', 'Ivern'],
        [4403325301, 'Fishlord', 467, 419, 'MIDDLE', 'CHALLENGER', 'I', 1105, True, False, 570, 'SummonerFlash', 'SummonerHaste', 'Swain'],
        [4403325301, 'ControlMageEGirl', 181, 165, 'BOTTOM', 'GRANDMASTER', 'I', 662, False, False, 169, 'SummonerFlash', 'SummonerHeal', 'Jhin'],
        [4403325301, 'A D', 117, 91, 'UTILITY', 'GRANDMASTER', 'I', 772, False, False, 651, 'SummonerFlash', 'SummonerExhaust', 'Senna'],
        [4403325301, 'Augustine', 151, 124, 'TOP', 'CHALLENGER', 'I', 884, False, False, 368, 'SummonerHaste', 'SummonerTeleport', 'Gwen'],
        [4403325301, 'Dragowski', 239, 217, 'JUNGLE', 'MASTER', 'I', 82, False, False, 139, 'SummonerSmite', 'SummonerFlash', 'Diana'],
        [4403325301, 'MasKa', 155, 132, 'MIDDLE', 'GRANDMASTER', 'I', 518, True, False, 60, 'SummonerFlash', 'SummonerDot', 'Akshan'],
        [4403325301, 'spam tab', 245, 184, 'BOTTOM', 'CHALLENGER', 'I', 1395, True, False, 292, 'SummonerFlash', 'SummonerHeal', 'Aphelios'],
        [4403325301, 'Twitch BTangLoL', 144, 117, 'UTILITY', 'GRANDMASTER', 'I', 538, False, True, 312, 'SummonerFlash', 'SummonerDot', 'Neeko']
    ]
champion_stats =    [
        ['Annie', 2, 7, 6, False, 144, 10, 19], 
        ['Ivern', 3, 3, 14, False, 21, 71, 16], 
        ['Swain', 6, 4, 5, False, 183, 46, 14],
        ['Jhin', 9, 4, 12, False, 202, 2, 22],
        ['Senna', 4, 6, 16, False, 34, 13, 77],
        ['Gwen', 7, 2, 1, True, 242, 19, 22],
        ['Diana', 3, 7, 7, True, 17, 50, 14],
        ['Akshan', 4, 6, 8, True, 159, 32, 14],
        ['Aphelios', 6, 4, 8, True, 211, 22, 29],
        ['Neeko', 4, 5, 8, True, 39, 17, 55]      
    ]
team_stats =  [
        4403325301, False,
        ['Garen', 'Sylas', 'Kindred', 'Aatrox', 'Draven'],
        ['Kalista', 'Zed', 'Fiora', 'Qiyana', 'Yuumi']   
    ]

In [365]:

with open('data/Games.csv', 'a') as f:
    thewriter = csv.writer(f)
    thewriter.writerow(Games)

In [362]:
with open('data/Players.csv', 'a') as f:
    thewriter = csv.writer(f)
    thewriter.writerow(Players)

In [363]:
with open('data/Champion_stats.csv', 'a') as f:
    thewriter = csv.writer(f)
    thewriter.writerow(champion_stats)

In [364]:
with open('data/Teams.csv', 'a') as f:
    thewriter = csv.writer(f)
    thewriter.writerow(team_stats)

In [366]:
hello = [1,2,3]
hello[0:2]

[1, 2]